<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/phi4_sim_2d_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# phi4_sim_2d.py
# Single-file 2D φ^4 field simulator with energy diagnostics and optional damping/absorption.

import os
import json
import math
import time
import argparse
from dataclasses import dataclass, asdict
from typing import Tuple, Literal

import numpy as np
import matplotlib.pyplot as plt


# =========================
# Configuration (defaults)
# =========================
@dataclass
class Config:
    # Grid / time
    nx: int = 256
    ny: int = 256
    dx: float = 1.0
    dt: float = 0.1
    steps: int = 5000

    # Physics
    c: float = 1.0           # wave speed
    lam: float = 1.0         # λ (self-interaction)
    v: float = 1.0           # vacuum expectation value
    eta: float = 0.0         # bulk damping coefficient
    absorb_width: int = 0    # width (cells) of edge absorption ramp (0 disables)
    absorb_eta: float = 0.1  # max edge damping

    # Initialization
    init: Literal["random", "blob", "bias"] = "random"
    random_amp: float = 0.01
    blob_sigma: float = 20.0
    bias_value: float = 0.9

    # Output
    out_dir: str = "out_phi4"
    save_every: int = 50
    cmap: str = "seismic"
    seed: int = 42

    # Numeric
    dtype: Literal["float32", "float64"] = "float64"


# =========================
# Utilities
# =========================
def ensure_dir(p: str) -> None:
    os.makedirs(p, exist_ok=True)

def save_json(path: str, obj) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)

def arr_dtype(cfg: Config):
    return np.float64 if str(cfg.dtype).lower() == "float64" else np.float32


# =========================
# Physics helpers
# =========================
def five_point_laplacian(phi: np.ndarray, dx: float) -> np.ndarray:
    return (
        np.roll(phi, +1, axis=0) +
        np.roll(phi, -1, axis=0) +
        np.roll(phi, +1, axis=1) +
        np.roll(phi, -1, axis=1) -
        4.0 * phi
    ) / (dx * dx)

def central_gradients(phi: np.ndarray, dx: float) -> Tuple[np.ndarray, np.ndarray]:
    gx = (np.roll(phi, -1, axis=0) - np.roll(phi, +1, axis=0)) / (2.0 * dx)
    gy = (np.roll(phi, -1, axis=1) - np.roll(phi, +1, axis=1)) / (2.0 * dx)
    return gx, gy

def potential(phi: np.ndarray, lam: float, v: float) -> np.ndarray:
    return 0.25 * lam * (phi * phi - v * v) ** 2

def dV_dphi(phi: np.ndarray, lam: float, v: float) -> np.ndarray:
    return lam * phi * (phi * phi - v * v)

def build_absorb_mask(nx: int, ny: int, width: int, max_eta: float, dtype) -> np.ndarray:
    if width <= 0 or max_eta <= 0:
        return np.zeros((nx, ny), dtype=dtype)
    i = np.arange(nx)
    j = np.arange(ny)
    ii, jj = np.meshgrid(i, j, indexing="ij")
    dist = np.minimum.reduce([ii, jj, nx - 1 - ii, ny - 1 - jj]).astype(dtype)
    ramp = np.clip(1.0 - dist / width, 0.0, 1.0)
    smooth = 0.5 * (1.0 - np.cos(np.pi * ramp))
    return (max_eta * smooth).astype(dtype)


# =========================
# Simulator
# =========================
class FieldSimulator:
    def __init__(self, cfg: Config):
        self.cfg = cfg
        self.nx, self.ny = cfg.nx, cfg.ny
        self.dx, self.dt = cfg.dx, cfg.dt
        self.c, self.lam, self.v = cfg.c, cfg.lam, cfg.v
        self.eta = cfg.eta

        self.dtype = arr_dtype(cfg)
        self.absorb = build_absorb_mask(self.nx, self.ny, cfg.absorb_width, cfg.absorb_eta, self.dtype)

        # Fields
        self.phi = np.zeros((self.nx, self.ny), dtype=self.dtype)
        self.phi_prev = np.zeros_like(self.phi)

        self._initialize()
        self._check_cfl()

    def _check_cfl(self):
        s = self.c * self.dt / self.dx
        s_max = 1.0 / math.sqrt(2.0)  # 2D 5-point guideline
        if s > s_max:
            print(f"[WARN] CFL condition likely violated: c*dt/dx = {s:.3f} > {s_max:.3f}")
        else:
            print(f"[INFO] CFL OK: c*dt/dx = {s:.3f} <= {s_max:.3f}")

    def _initialize(self):
        rng = np.random.default_rng(self.cfg.seed)
        if self.cfg.init == "random":
            self.phi[:] = self.cfg.random_amp * (rng.random((self.nx, self.ny)) - 0.5)
        elif self.cfg.init == "blob":
            i = np.arange(self.nx)
            j = np.arange(self.ny)
            ii, jj = np.meshgrid(i, j, indexing="ij")
            r2 = (ii - (self.nx - 1) / 2.0) ** 2 + (jj - (self.ny - 1) / 2.0) ** 2
            self.phi[:] = np.exp(-r2 / (2.0 * self.cfg.blob_sigma * self.cfg.blob_sigma))
        elif self.cfg.init == "bias":
            self.phi[:] = self.cfg.bias_value
        else:
            raise ValueError(f"Unknown init: {self.cfg.init}")
        self.phi_prev[:] = self.phi

    def step(self):
        lap = five_point_laplacian(self.phi, self.dx)
        force = (self.c * self.c) * lap - dV_dphi(self.phi, self.lam, self.v)

        eta_eff = self.eta + self.absorb
        damp = eta_eff * self.dt
        phi_new = ((2.0 - damp) * self.phi) - ((1.0 - damp) * self.phi_prev) + (self.dt * self.dt) * force

        self.phi_prev, self.phi = self.phi, phi_new

    def energy_components(self) -> Tuple[float, float, float]:
        vel = (self.phi - self.phi_prev) / self.dt
        Ek = 0.5 * (vel * vel)

        gx, gy = central_gradients(self.phi, self.dx)
        Eg = 0.5 * (self.c * self.c) * (gx * gx + gy * gy)

        Ep = potential(self.phi, self.lam, self.v)

        cell = self.dx * self.dx
        return float(Ek.sum() * cell), float(Eg.sum() * cell), float(Ep.sum() * cell)

    def total_energy(self) -> float:
        Ek, Eg, Ep = self.energy_components()
        return Ek + Eg + Ep


# =========================
# Analytics / visualization
# =========================
def domain_wall_length(phi: np.ndarray, dx: float) -> float:
    s = np.sign(phi)
    s[s == 0] = 1
    changes_x = (s != np.roll(s, -1, axis=0)).sum()
    changes_y = (s != np.roll(s, -1, axis=1)).sum()
    length = 0.5 * (changes_x + changes_y) * dx
    return float(length)

def plot_field(phi: np.ndarray, step: int, out_dir: str, cmap: str = "seismic"):
    ensure_dir(out_dir)
    plt.figure(figsize=(6, 6))
    im = plt.imshow(phi.T, cmap=cmap, origin="lower")
    plt.colorbar(im, fraction=0.046, pad=0.04, label="φ")
    plt.title(f"Field φ at step {step}")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.tight_layout()
    path = os.path.join(out_dir, f"field_{step:06d}.png")
    plt.savefig(path, dpi=120)
    plt.close()


# =========================
# Runner
# =========================
def run(cfg: Config):
    out = cfg.out_dir
    frames_dir = os.path.join(out, "frames")
    ensure_dir(out)
    ensure_dir(frames_dir)

    save_json(os.path.join(out, "config.json"), asdict(cfg))

    np.random.seed(cfg.seed)
    sim = FieldSimulator(cfg)

    energy_path = os.path.join(out, "energy.csv")
    with open(energy_path, "w", encoding="utf-8") as f:
        f.write("step,energy,Ek,Eg,Ep,wall_length\n")

    print("[INFO] Starting simulation...")
    t0 = time.time()
    for step in range(cfg.steps + 1):
        if step % cfg.save_every == 0:
            Ek, Eg, Ep = sim.energy_components()
            E = Ek + Eg + Ep
            L = domain_wall_length(sim.phi, cfg.dx)
            with open(energy_path, "a", encoding="utf-8") as f:
                f.write(f"{step},{E:.6e},{Ek:.6e},{Eg:.6e},{Ep:.6e},{L:.6e}\n")
            plot_field(sim.phi, step, frames_dir, cfg.cmap)
            print(f"Step {step:6d} | Energy = {E:.6e}")

        if step < cfg.steps:
            sim.step()

    dt = time.time() - t0
    print(f"[DONE] Completed {cfg.steps} steps in {dt:.2f}s. Outputs in: {out}")


# =========================
# CLI (notebook-safe)
# =========================
def build_parser():
    p = argparse.ArgumentParser(
        description="Single-file 2D φ^4 field simulator with leapfrog integration, energy diagnostics, and optional damping/absorption."
    )
    # Grid/time
    p.add_argument("--nx", type=int)
    p.add_argument("--ny", type=int)
    p.add_argument("--dx", type=float)
    p.add_argument("--dt", type=float)
    p.add_argument("--steps", type=int)

    # Physics
    p.add_argument("--c", type=float)
    p.add_argument("--lam", type=float)
    p.add_argument("--v", type=float)
    p.add_argument("--eta", type=float)
    p.add_argument("--absorb_width", type=int)
    p.add_argument("--absorb_eta", type=float)

    # Init
    p.add_argument("--init", choices=["random", "blob", "bias"])
    p.add_argument("--random_amp", type=float)
    p.add_argument("--blob_sigma", type=float)
    p.add_argument("--bias_value", type=float)

    # Output
    p.add_argument("--out_dir", type=str)
    p.add_argument("--save_every", type=int)
    p.add_argument("--cmap", type=str)
    p.add_argument("--seed", type=int)

    # Numeric
    p.add_argument("--dtype", choices=["float32", "float64"])
    return p

def merge_args_into_cfg(args, cfg: Config) -> Config:
    d = asdict(cfg)
    for k, v in vars(args).items():
        if v is not None and k in d:
            d[k] = v
    return Config(**d)

def main(argv=None):
    parser = build_parser()
    args, _ = parser.parse_known_args(argv)  # notebook-safe
    cfg = merge_args_into_cfg(args, Config())
    run(cfg)

if __name__ == "__main__":
    main()